# Preprocessamento

Vamos trabalhar com:

* Dados Ausentes
* Dados Categoricos



#### Imports

In [92]:
%pylab inline
from io import StringIO
import pandas as pd

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

Populating the interactive namespace from numpy and matplotlib


## Lidando com Dados Ausentes - **Removendo Exemplos ou Features**

#### Carregando e visualizando o data-frame

In [93]:
#simulando como se estivessemos abrindo um arquivo csv
csv_data = '''Atrib1,Atrib2,Atrib3,Atrib4
11.1,21.2,33.3,40.4
15.5,26.0,,48.0
16.0,21.0,32.2'''
df = pd.read_csv(StringIO(csv_data))
df

,Atrib1,Atrib2,Atrib3,Atrib4
0,11.1,21.2,33.3,40.4
1,15.5,26.0,NaN,48.0
2,16.0,21.0,32.2,NaN


Mais algumas informacoes sobre o dataset:

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Atrib1  3 non-null      float64
 1   Atrib2  3 non-null      float64
 2   Atrib3  2 non-null      float64
 3   Atrib4  2 non-null      float64
dtypes: float64(4)
memory usage: 228.0 bytes


#### Procurando dados faltantes com `isna()` ou `isnull()`

In [95]:
df.isna()

,Atrib1,Atrib2,Atrib3,Atrib4
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


#### Contando dados faltantes com `sum()`

In [96]:
df.isna().sum()

,0
Atrib1,0
Atrib2,0
Atrib3,1
Atrib4,1


#### Removendo os exemplos (linhas) contendo **algum** dado faltante
(Nao atualize o dataframe)

In [97]:
df.dropna()

,Atrib1,Atrib2,Atrib3,Atrib4
0,11.1,21.2,33.3,40.4


#### Removendo as features (colunas) contendo **algum** dado faltante
(Nao atualize o dataframe)

In [98]:
df.dropna(axis=1)

,Atrib1,Atrib2
0,11.1,21.2
1,15.5,26.0
2,16.0,21.0


#### Vamos mudar um pouco o data-frame

In [99]:
csv_data = '''Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
11.1,21.2,33.3,40.4
15.5,26.0,,48.0
16.0,21.0,32.2
,,,,'''
df = pd.read_csv(StringIO(csv_data))
df

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,NaN
2,16.0,21.0,32.2,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN


#### Removendo apenas os exemplos onde **todas** as features estao sem valor

(Nao atualize o dataframe)

In [100]:
df.dropna(how="all")

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,NaN
2,16.0,21.0,32.2,NaN,NaN


#### Removendo apenas as features onde **todos** os exemplos estao sem valor

(Nao atualize o dataframe)

In [101]:
df.dropna(axis=1, how="all")

,Atrib1,Atrib2,Atrib3,Atrib4
0,11.1,21.2,33.3,40.4
1,15.5,26.0,NaN,48.0
2,16.0,21.0,32.2,NaN
3,NaN,NaN,NaN,NaN


#### Removendo os exemplos contendo valores validos em menos de 3 features

(Nao atualize o dataframe)

In [102]:
#Remove as linhas com menos de 3 valores validos (non-NaN)
#Permanece de tiver 3 ou mais valores validos

df.dropna(thresh=3)

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,NaN
2,16.0,21.0,32.2,NaN,NaN


#### Removendo os exemplos contendo 2 ou mais valores faltantes

(Nao atualize o dataframe)

In [103]:
#Se deve remover com 2 NaN ou mais, o limite aceitavel de NaN eh 1

df.dropna(thresh=df.columns.size-1)

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN


#### Remover exemplos onde Atrib2 **ou** Atrib4 estejam sem valor

In [104]:
df.dropna(subset=["Atrib2", "Atrib4"])

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,NaN


#### Remover exemplos onde Atrib2 **e** Atrib4 estejam sem valor

In [105]:
df.dropna(how="all", subset=["Atrib2", "Atrib4"])

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,NaN
2,16.0,21.0,32.2,NaN,NaN


## Lidando com Dados Ausentes - **Adicionando Dados**

#### Visualizando o data-frame

In [106]:
df

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,NaN
2,16.0,21.0,32.2,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN


#### Adicionando dados com `SimpleImputer` (Exemplo 1)

Vamos entender a estrutura dos Transformers

Vamos usar o transformer `SimpleImputer` para substituir valores faltantes pela media

In [107]:
imputer = SimpleImputer(strategy='mean')
imputer = imputer.fit(df)
imputed_data = imputer.transform(df.values)
imputed_data

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['Atrib5']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


array([[11.1       , 21.2       , 33.3       , 40.4       ],
       [15.5       , 26.        , 32.75      , 48.        ],
       [16.        , 21.        , 32.2       , 44.2       ],
       [14.2       , 22.73333333, 32.75      , 44.2       ]])

#### Gerando um data-frame um pouco diferente

In [108]:
csv_data = '''Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
11.1,21.2,33.3,40.4,999
15.5,26.0,999,48.0,80.1
16.0,21.0,32.2,81.1,999
13.3,22.0,31.2,999,999'''
df = pd.read_csv(StringIO(csv_data))
df

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,999.0
1,15.5,26.0,999.0,48.0,80.1
2,16.0,21.0,32.2,81.1,999.0
3,13.3,22.0,31.2,999.0,999.0


#### Adicionando dados com `SimpleImputer` (Exemplo 2)

Valores faltantes sao representados por 999

Valores faltantes devem ser substituidos pela mediana

In [109]:
imputer = SimpleImputer(missing_values=999, strategy='median')
imputer = imputer.fit(df)
imputed_data = imputer.transform(df.values)
imputed_data

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[11.1, 21.2, 33.3, 40.4, 80.1],
       [15.5, 26. , 32.2, 48. , 80.1],
       [16. , 21. , 32.2, 81.1, 80.1],
       [13.3, 22. , 31.2, 48. , 80.1]])

#### Adicionando dados com `SimpleImputer` (Exemplo 3)

Valores faltantes sao representados por 999

Usar SimpleImputer para reverter valores faltantes para NaN e atualizar data-frame

In [110]:
imputer = SimpleImputer(missing_values=999, strategy='constant', fill_value=np.nan)
imputer = imputer.fit(df)
imputed_data = imputer.transform(df.values)
pd.DataFrame(imputed_data)

#se quiser alterar o data-frame
#df = pd.DataFrame(imputed_data)
#df

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


,0,1,2,3,4
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,80.1
2,16.0,21.0,32.2,81.1,NaN
3,13.3,22.0,31.2,NaN,NaN


Observe que isto seria equivalente a usar `df.replace()`

In [111]:
df.replace(999., np.nan)

,Atrib1,Atrib2,Atrib3,Atrib4,Atrib5
0,11.1,21.2,33.3,40.4,NaN
1,15.5,26.0,NaN,48.0,80.1
2,16.0,21.0,32.2,81.1,NaN
3,13.3,22.0,31.2,NaN,NaN


## Lidando com Dados Categoricos

#### Criando um data-frame contendo valores categoricos

In [112]:
df_backup = pd.DataFrame([
['amarelo', 'GG', 79.90, 'classe1'],
['azul', 'M', 49.50, 'classe2'],
['preto', 'P', 54.30, 'classe3'],
['preto', 'PP', 65.00, 'classe2'],
['branco', 'M', 69.90, 'classe3'],
['cinza', 'G', 75.90, 'classe2'],
['marrom', 'GG', 59.80, 'classe3']])

df_backup.columns = ['cor', 'tamanho', 'preco', 'classe']

df = df_backup.copy() # podemos restaurar esse backup desta forma se baguncarmos muito nosso df

df #vamos usar este data-frame nos exemplos

,cor,tamanho,preco,classe
0,amarelo,GG,79.9,classe1
1,azul,M,49.5,classe2
2,preto,P,54.3,classe3
3,preto,PP,65.0,classe2
4,branco,M,69.9,classe3
5,cinza,G,75.9,classe2
6,marrom,GG,59.8,classe3


#### Explorando informacoes sobre **valores unicos**

Verificando a **quantidade** de valores unicos

In [113]:
categorical_cols = ['cor', 'tamanho', 'classe']
df[categorical_cols].apply(lambda x: x.nunique(), axis=0)

,0
cor,6
tamanho,5
classe,3


Observando os valores unicos

In [114]:
for cat in categorical_cols:
  print(cat, df[cat].unique())

cor ['amarelo' 'azul' 'preto' 'branco' 'cinza' 'marrom']
tamanho ['GG' 'M' 'P' 'PP' 'G']
classe ['classe1' 'classe2' 'classe3']


### Fazendo mapeamentos com dicionarios (`dict`)

#### Mapeamentos com `dict` - Exemplo 1

Convertendo o atributo `tamanho` em inteiros

In [115]:
tam_map = {'PP': 0, 'P': 1, 'M': 2, 'G': 3, 'GG': 4}
df['tamanho'] = df['tamanho'].map(tam_map)
df

,cor,tamanho,preco,classe
0,amarelo,4,79.9,classe1
1,azul,2,49.5,classe2
2,preto,1,54.3,classe3
3,preto,0,65.0,classe2
4,branco,2,69.9,classe3
5,cinza,3,75.9,classe2
6,marrom,4,59.8,classe3


#### Mapeamentos com `dict` - Exemplo 2

Revertendo os valores categoricos (atributo `tamanho`)

In [116]:
#criacao do dict para mapeamento reverso

inv_tam_map = {v: k for k, v in tam_map.items()}

#imprimindo ambos para comparar

original_sorted = sorted(tam_map.items(), key=lambda x:x[1]) #original ordenado pelos valores

print ('Original: {0}'.format(original_sorted))
print ('Invertido: {0}'.format(inv_tam_map))

Original: [('PP', 0), ('P', 1), ('M', 2), ('G', 3), ('GG', 4)]
Invertido: {0: 'PP', 1: 'P', 2: 'M', 3: 'G', 4: 'GG'}


In [117]:
#alteracao do data-frame

df['tamanho'] = df["tamanho"].map(inv_tam_map)
df

,cor,tamanho,preco,classe
0,amarelo,GG,79.9,classe1
1,azul,M,49.5,classe2
2,preto,P,54.3,classe3
3,preto,PP,65.0,classe2
4,branco,M,69.9,classe3
5,cinza,G,75.9,classe2
6,marrom,GG,59.8,classe3


#### Mapeamentos com `dict` - Exemplo 3

Convertendo os rotulos das **classes** em inteiros

In [118]:
# observando as classes
np.unique(df['classe'])

array(['classe1', 'classe2', 'classe3'], dtype=object)

In [119]:
# como nao ha ordem, criamos o mapa (dict) atraves de um enumerate dos rotulos observados
classe_map = {label:idx for idx,label in
                 enumerate(np.unique(df['classe']))}
classe_map

{'classe1': 0, 'classe2': 1, 'classe3': 2}

In [120]:
# alteracao do data-frame
df['classe'] = df['classe'].map(classe_map)
df

,cor,tamanho,preco,classe
0,amarelo,GG,79.9,0
1,azul,M,49.5,1
2,preto,P,54.3,2
3,preto,PP,65.0,1
4,branco,M,69.9,2
5,cinza,G,75.9,1
6,marrom,GG,59.8,2


In [121]:
# revertendo para os valores originais (rotulos)
inv_classe_map = {v : k for k, v in classe_map.items()}
df['classe'] = df['classe'].map(inv_classe_map)
df

,cor,tamanho,preco,classe
0,amarelo,GG,79.9,classe1
1,azul,M,49.5,classe2
2,preto,P,54.3,classe3
3,preto,PP,65.0,classe2
4,branco,M,69.9,classe3
5,cinza,G,75.9,classe2
6,marrom,GG,59.8,classe3


### Fazendo mapeamentos com Encoders

#### Carregando o data-frame

In [122]:
# restaurando o data-frame original
df = df_backup.copy()
df

,cor,tamanho,preco,classe
0,amarelo,GG,79.9,classe1
1,azul,M,49.5,classe2
2,preto,P,54.3,classe3
3,preto,PP,65.0,classe2
4,branco,M,69.9,classe3
5,cinza,G,75.9,classe2
6,marrom,GG,59.8,classe3


#### Usando Encoders (Exemplo 1)

Codificando as **classes** com `LabelEncoder`

In [123]:
# fazendo fit a tranform em um unico comando
class_label_encoder = LabelEncoder()
y = class_label_encoder.fit_transform(df['classe'].values)
y

array([0, 1, 2, 1, 2, 1, 2])

In [124]:
#revertendo a codificacao
class_label_encoder.inverse_transform(y)

array(['classe1', 'classe2', 'classe3', 'classe2', 'classe3', 'classe2',
       'classe3'], dtype=object)

#### Usando Encoders (Exemplo 2)

Codificando as **cores** com `LabelEncoder`

LabelEncoder foi criado para codificar os rotulos das classes. Porem, como esses rotulos sao considerados Nominais / Sem Ordem, podemos usa-los para codificar outros atributos nominais.

Vamos codificar os valores do atributo `cor`.

**Restaurando e observando** o data-frame:

In [125]:
# restaurando o data-frame original
df = df_backup.copy()
df


,cor,tamanho,preco,classe
0,amarelo,GG,79.9,classe1
1,azul,M,49.5,classe2
2,preto,P,54.3,classe3
3,preto,PP,65.0,classe2
4,branco,M,69.9,classe3
5,cinza,G,75.9,classe2
6,marrom,GG,59.8,classe3


**Executando** o `LabelEncoder`

In [126]:
#codificando e alterando o data-frame

class_label_encoder = LabelEncoder()
y = class_label_encoder.fit_transform(df['classe'].values)
y

array([0, 1, 2, 1, 2, 1, 2])

**Verificando** o `df` modificado

In [127]:
df

,cor,tamanho,preco,classe
0,amarelo,GG,79.9,classe1
1,azul,M,49.5,classe2
2,preto,P,54.3,classe3
3,preto,PP,65.0,classe2
4,branco,M,69.9,classe3
5,cinza,G,75.9,classe2
6,marrom,GG,59.8,classe3


#### Usando Encoders (Exemplo 3)

Codificando as **cores** com `LabelEncoder`

Vamos ver **outra forma** de fazer: manipulando os arrays numpy.

**Restaurando e observando** o data-frame:

In [128]:
# restaurando o data-frame original
df = df_backup.copy()

#matriz de features
X = df[['cor', 'tamanho', 'preco']].values
X

array([['amarelo', 'GG', 79.9],
       ['azul', 'M', 49.5],
       ['preto', 'P', 54.3],
       ['preto', 'PP', 65.0],
       ['branco', 'M', 69.9],
       ['cinza', 'G', 75.9],
       ['marrom', 'GG', 59.8]], dtype=object)

**Executando** o `LabelEncoder`

In [129]:
#alterando df['cor']
cor_encoder = LabelEncoder()

#alterando a matriz de features
X[:, 0] = cor_encoder.fit_transform(X[:, 0])

#atualizando o data-frame
df['cor'] = X[:, 0]


**Verificando** o `df` modificado

In [130]:
df

,cor,tamanho,preco,classe
0,0,GG,79.9,classe1
1,1,M,49.5,classe2
2,5,P,54.3,classe3
3,5,PP,65.0,classe2
4,2,M,69.9,classe3
5,3,G,75.9,classe2
6,4,GG,59.8,classe3


#### Usando Encoders (Exemplo 4)

**NAO** devemos usar `LabelEncoder` para codificar o atributo `tamanho`, pois ele eh **ORDINAL**.

Vamos codificando o atributo `tamanho` com `OrdinalEncoder`

**Restaurando e observando** o data-frame:

In [131]:
# restaurando o data-frame original
df = df_backup.copy()

#X = df[['cor', 'tamanho', 'preco']].values
#X

**Executando** o `OrdinalEncoder`

In [132]:
tamanho_encoder = OrdinalEncoder()
df.tamanho = tamanho_encoder.fit_transform(df.tamanho.values.reshape(-1,1))
df

,cor,tamanho,preco,classe
0,amarelo,1.0,79.9,classe1
1,azul,2.0,49.5,classe2
2,preto,3.0,54.3,classe3
3,preto,4.0,65.0,classe2
4,branco,2.0,69.9,classe3
5,cinza,0.0,75.9,classe2
6,marrom,1.0,59.8,classe3


Vamos usar o **parametro** `categories=[[]]` para **garantir a ordem** (especificar os valores das features em ordem).

In [133]:
#restaurando o df
df = df_backup.copy()

#instanciando o encoder com parametro categories
tamanho_encoder = OrdinalEncoder(categories=[['PP', 'P', 'M', 'G', 'GG']])

#codificando e visualizando df
df.tamanho = tamanho_encoder.fit_transform(df.tamanho.values.reshape(-1,1))
df

,cor,tamanho,preco,classe
0,amarelo,4.0,79.9,classe1
1,azul,2.0,49.5,classe2
2,preto,1.0,54.3,classe3
3,preto,0.0,65.0,classe2
4,branco,2.0,69.9,classe3
5,cinza,3.0,75.9,classe2
6,marrom,4.0,59.8,classe3


**Outra forma** de fazer

In [134]:
df = df_backup.copy()
X = df[['cor', 'tamanho', 'preco']].values
tamanho_encoder = OrdinalEncoder(categories=[['PP', 'P', 'M', 'G', 'GG']])
X[:, 1] = tamanho_encoder.fit_transform(df[['tamanho']].values)[:,0]
df['tamanho'] = X[:, 1]
df

,cor,tamanho,preco,classe
0,amarelo,4.0,79.9,classe1
1,azul,2.0,49.5,classe2
2,preto,1.0,54.3,classe3
3,preto,0.0,65.0,classe2
4,branco,2.0,69.9,classe3
5,cinza,3.0,75.9,classe2
6,marrom,4.0,59.8,classe3


#### Usando Encoders - Exemplo 5

(funciona em outras colunas, mas eh bom lembrar que foi projetado para a variavel target)

**Executando** `LabelEncoder` no **data-frame inteiro**

In [135]:
df = df_backup.copy() #restaurando o df original
le = LabelEncoder() #instanciando o encoder
df = df.apply(le.fit_transform)
df

,cor,tamanho,preco,classe
0,0,1,6,0
1,1,2,0,1
2,5,3,1,2
3,5,4,3,1
4,2,2,4,2
5,3,0,5,1
6,4,1,2,2


**Executando** `LabelEncoder` em **multiplas features**


In [136]:
df = df_backup.copy() #restaurando o df original
le = LabelEncoder() #instanciando o encoder
categorical_cols = ['cor', 'tamanho']
df[categorical_cols] = df[categorical_cols].apply(le.fit_transform)
df

,cor,tamanho,preco,classe
0,0,1,79.9,classe1
1,1,2,49.5,classe2
2,5,3,54.3,classe3
3,5,4,65.0,classe2
4,2,2,69.9,classe3
5,3,0,75.9,classe2
6,4,1,59.8,classe3


#### Usando Encoders - Exemplo 6

**Executando** `OneHotEncoder` nas features **cor** e **tamanho** (apenas para ilustrar, pois tamanho eh ordinal)

In [137]:
df = df_backup.copy() #restaurando o df original
X = df[['cor','tamanho']].values
ohe = OneHotEncoder(sparse_output=False)
ohe.fit_transform(X)

array([[1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

#### Usando Encoders - Exemplo 7

**Executando** `LabelBinarizer` na **classe**

In [138]:
df = df_backup.copy() #restaurando o df original
X = df[['classe']].values
#X = df[['cor','tamanho']].values # nao suportado - erro
lb = LabelBinarizer()
lb.fit_transform(X)

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

#### Usando Encoders - Exemplo 8

Utilizando `ColumnTransformer` para compor **diferentes Encoders**

In [139]:
df = df_backup.copy() #restaurando o df original
columnTransformer = ColumnTransformer(
    [
     ('nominal', OneHotEncoder(sparse_output=False), [0]),
     ('ordinal', OrdinalEncoder(categories=[['PP', 'P', 'M', 'G', 'GG']]), [1]),
     ],
    remainder='passthrough')
columnTransformer.fit_transform(df)

#pd.DataFrame(columnTransformer.fit_transform(df))


array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 79.9, 'classe1'],
       [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 49.5, 'classe2'],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 54.3, 'classe3'],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 65.0, 'classe2'],
       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 69.9, 'classe3'],
       [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 75.9, 'classe2'],
       [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 59.8, 'classe3']], dtype=object)

Utilizando `make_column_transformer` para compor **diferentes Encoders**





In [140]:
df = df_backup.copy() #restaurando o df original
preprocess = make_column_transformer(
    (OneHotEncoder(sparse_output=False),['cor']),
    (OrdinalEncoder(),['tamanho']),
    remainder='passthrough')
preprocess.fit_transform(df)

array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 79.9, 'classe1'],
       [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 49.5, 'classe2'],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 54.3, 'classe3'],
       [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 4.0, 65.0, 'classe2'],
       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 69.9, 'classe3'],
       [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 75.9, 'classe2'],
       [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 59.8, 'classe3']], dtype=object)

### Pandas `get_dummies`

Para mais detalhes, veja a [documentacao](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html).

In [141]:
df = df_backup.copy()
pd.get_dummies(df)

,preco,cor_amarelo,cor_azul,cor_branco,cor_cinza,cor_marrom,cor_preto,tamanho_G,tamanho_GG,tamanho_M,tamanho_P,tamanho_PP,classe_classe1,classe_classe2,classe_classe3
0,79.9,True,False,False,False,False,False,False,True,False,False,False,True,False,False
1,49.5,False,True,False,False,False,False,False,False,True,False,False,False,True,False
2,54.3,False,False,False,False,False,True,False,False,False,True,False,False,False,True
3,65.0,False,False,False,False,False,True,False,False,False,False,True,False,True,False
4,69.9,False,False,True,False,False,False,False,False,True,False,False,False,False,True
5,75.9,False,False,False,True,False,False,True,False,False,False,False,False,True,False
6,59.8,False,False,False,False,True,False,False,True,False,False,False,False,False,True


In [142]:
df = df_backup.copy()
pd.get_dummies(df[['cor', 'tamanho']])

,cor_amarelo,cor_azul,cor_branco,cor_cinza,cor_marrom,cor_preto,tamanho_G,tamanho_GG,tamanho_M,tamanho_P,tamanho_PP
0,True,False,False,False,False,False,False,True,False,False,False
1,False,True,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,True,False,False,False,True,False
3,False,False,False,False,False,True,False,False,False,False,True
4,False,False,True,False,False,False,False,False,True,False,False
5,False,False,False,True,False,False,True,False,False,False,False
6,False,False,False,False,True,False,False,True,False,False,False


In [143]:
df = df_backup.copy()
for column in ['cor', 'tamanho']:
    dummies = pd.get_dummies(df[column])
    df[dummies.columns] = dummies
    df = df.drop([column], axis=1)
df

,preco,classe,amarelo,azul,branco,cinza,marrom,preto,G,GG,M,P,PP
0,79.9,classe1,True,False,False,False,False,False,False,True,False,False,False
1,49.5,classe2,False,True,False,False,False,False,False,False,True,False,False
2,54.3,classe3,False,False,False,False,False,True,False,False,False,True,False
3,65.0,classe2,False,False,False,False,False,True,False,False,False,False,True
4,69.9,classe3,False,False,True,False,False,False,False,False,True,False,False
5,75.9,classe2,False,False,False,True,False,False,True,False,False,False,False
6,59.8,classe3,False,False,False,False,True,False,False,True,False,False,False


In [144]:
df = df_backup.copy()
pd.get_dummies(data=df, columns=['cor', 'tamanho'])


,preco,classe,cor_amarelo,cor_azul,cor_branco,cor_cinza,cor_marrom,cor_preto,tamanho_G,tamanho_GG,tamanho_M,tamanho_P,tamanho_PP
0,79.9,classe1,True,False,False,False,False,False,False,True,False,False,False
1,49.5,classe2,False,True,False,False,False,False,False,False,True,False,False
2,54.3,classe3,False,False,False,False,False,True,False,False,False,True,False
3,65.0,classe2,False,False,False,False,False,True,False,False,False,False,True
4,69.9,classe3,False,False,True,False,False,False,False,False,True,False,False
5,75.9,classe2,False,False,False,True,False,False,True,False,False,False,False
6,59.8,classe3,False,False,False,False,True,False,False,True,False,False,False


## Alternativas ao scikit-learn para preprocessamento

Feature Engine: [Documentacao](https://feature-engine.readthedocs.io/en/latest/index.html)  |  [Github](https://github.com/solegalli/feature_engine)

Category Encoders: [Documentacao](https://github.com/scikit-learn-contrib/category_encoders)

StatModels - Pasty: [Documentacao](https://www.statsmodels.org/devel/contrasts.html)

Sklearn-pandas: [Github](https://github.com/scikit-learn-contrib/sklearn-pandas)
